In [1]:
import pandas as pd
from pyvi import ViTokenizer
import re
import nltk
import seaborn as sns
from matplotlib import pyplot
import numpy as np

# 1. Import data

In [2]:
train_filename = "train_nor_811.xlsx"
valid_filename = "valid_nor_811.xlsx"
test_filename = "test_nor_811.xlsx"
train_data = pd.read_excel(train_filename, engine = "openpyxl")
valid_data = pd.read_excel(valid_filename, engine = "openpyxl")
test_data = pd.read_excel(test_filename, engine = "openpyxl")

In [3]:
from sklearn.preprocessing import LabelEncoder
def file_processing(data):
    data.drop(columns = {"Unnamed: 0"}, axis = 1, inplace = True)
    data["emotion_encode"] = data["Emotion"]
    encoder = LabelEncoder()
    data.emotion_encode = encoder.fit_transform(data.Emotion)
    return data

In [4]:
train_data = file_processing(train_data)
valid_data = file_processing(valid_data)
test_data = file_processing(test_data)

# 2. Data visualization

# 3. Data preprocessing

In [5]:
def remove_duplicate(word):
    prev_char = ""
    clean_word = ""
    for character in word:
        if(character != prev_char):
            clean_word += character
            prev_char = character
    return clean_word

In [6]:
def deEmojify(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def normalize_sentences(sentences):
    punc_lst = {'.', ',', '...', '-', '“', '”', ':', '(', ')', '"', '!', '&', ';', '?', '*', ']', '>', '…', '’',"``","''", "=", "%", "^", "@", "<", ">"}
    confusing_words = {"per"}
    acronym_word = {
        "ko" : "không",
        "k" : "không",
        "z" : "vậy",
        "v" : "vậy",
        "dzậy" : "vậy",
        "dậy": "vậy",
        "t" : "tao",
        "m" : "mày",
        "sgk" : "sách_giáo_khoa",
        "zi" : "vậy",
        "dth" : "dễ_thương",
        "dume": "đụ mẹ"
    }
    
    clean_sentences = []
    
    # remove punctuation and lowercase
    for sent in sentences:
        
        # remove emojis
        sent = deEmojify(sent)
        
        sent = nltk.word_tokenize(sent)
        temp = []
        for word in sent:
            word = word.lower()
            word = remove_duplicate(word)
            if (word in punc_lst or word in confusing_words):
                continue
            elif(word in acronym_word):
                temp.append(acronym_word[word])
            elif(word.isdigit()):
                temp.append("<NUM>")
            else:
                temp.append(word)
        # remove whitespace
        sent = ' '.join(temp)
        
        clean_sentences.append(sent)
        
    return clean_sentences

In [7]:
def normalize_dataset(data):
    sentences = []
    for i in range(len(data)):
        sentences.append(ViTokenizer.tokenize(data.Sentence[i]))
    
    sentences = normalize_sentences(sentences)
    encode_tags = data.emotion_encode
    
    # remove empty sentences
    for idx, sent in enumerate(sentences):
        if sent.strip() == "":
            del sentences[idx]
            del encode_tags[idx]
    
    return sentences, encode_tags

In [8]:
train_clean_sentences, train_encode_tags = normalize_dataset(train_data)
valid_clean_sentences, valid_encode_tags = normalize_dataset(valid_data)
test_clean_sentences, test_encode_tags = normalize_dataset(test_data)

# 4. Model architecture

In [9]:
from tensorflow.keras.layers import Flatten
from tensorflow.keras import layers, activations , models , preprocessing , utils
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

### Tokenizer

In [10]:
MAX_LEN = 150

tokenizer = Tokenizer(lower = True, split = ' ')
tokenizer.fit_on_texts(train_clean_sentences + valid_clean_sentences)

X_train = tokenizer.texts_to_sequences(train_clean_sentences)
X_train = pad_sequences(X_train, MAX_LEN, padding='post', truncating='post')

X_valid = tokenizer.texts_to_sequences(valid_clean_sentences)
X_valid = pad_sequences(X_valid, MAX_LEN, padding='post', truncating='post')

X_test = tokenizer.texts_to_sequences(test_clean_sentences)
X_test = pad_sequences(X_test, MAX_LEN, padding='post', truncating='post')

In [11]:
vocab_size = len(tokenizer.word_index) + 1

### Word2Vec

In [108]:
from gensim.models import Word2Vec
import numpy as np
MAX_LEN = 150
w2v_model = Word2Vec(train_clean_sentences + valid_clean_sentences, min_count = 1, size = MAX_LEN)

In [109]:
def convert2vec(sentence):
    _sum = np.array([0]*MAX_LEN)
    for word in sentence:
        if not(word in w2v_model.wv.vocab):
            continue
        vec = w2v_model.wv[word]
        _sum = _sum + vec
    return _sum/len(sentence)

In [110]:
X_train = []
for sent in train_clean_sentences:
    if len(sent) == 0:
        X_train.append(np.array([0]*MAX_LEN))
    else:
        X_train.append(convert2vec(sent))
X_train = np.array(X_train)

X_valid = []
for sent in valid_clean_sentences:
    if len(sent) == 0:
        X_valid.append(np.array([0]*MAX_LEN))
    else:
        X_valid.append(convert2vec(sent))
X_valid = np.array(X_valid)

### TF IDF

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(use_idf = True, max_features = 3000)

fitted_vectorizer = tfidf_vectorizer.fit(train_clean_sentences)

X_train = fitted_vectorizer.transform(train_clean_sentences).toarray()

X_valid = fitted_vectorizer.transform(valid_clean_sentences).toarray()

X_test = fitted_vectorizer.transform(test_clean_sentences).toarray()

## 4.1 CNN Model

In [17]:
embedding_dim = 128

inputs = layers.Input(shape=( MAX_LEN , ))
embedding = layers.Embedding(vocab_size, embedding_dim, input_length=MAX_LEN)(inputs)

cnn1 = layers.Conv1D(filters=100, kernel_size=1, activation='relu')(embedding)
cnn1 = layers.MaxPooling1D(pool_size=2)(cnn1)
cnn1 = Flatten()(cnn1)

cnn2 = layers.Conv1D(filters=100, kernel_size=2, activation='relu')(embedding)
cnn2 = layers.MaxPooling1D(pool_size=2)(cnn2)
cnn2 = Flatten()(cnn2)

outputs = layers.Concatenate()([cnn1,cnn2])

outputs = layers.Dense(28, activation='tanh')(outputs)
outputs = layers.Dense(14, activation='tanh')(outputs)
outputs = layers.Dense(7, activation='softmax')(outputs)
model=models.Model(inputs,outputs)
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
# model.summary()

In [11]:
y_train = tf.keras.utils.to_categorical(train_encode_tags)
y_valid = tf.keras.utils.to_categorical(valid_encode_tags)

In [19]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1)
history = model.fit(X_train, y_train,
                    epochs = 10,
                    callbacks = [es],
                    validation_data=(X_valid, y_valid),
                    batch_size = 32)

Epoch 1/10
174/174 [==============================] - 131s 751ms/step - loss: 0.3900 - accuracy: 0.2627 - val_loss: 0.3767 - val_accuracy: 0.3120
Epoch 2/10
174/174 [==============================] - 127s 730ms/step - loss: 0.3825 - accuracy: 0.2809 - val_loss: 0.3759 - val_accuracy: 0.3120
Epoch 3/10
174/174 [==============================] - 126s 725ms/step - loss: 0.3822 - accuracy: 0.2809 - val_loss: 0.3760 - val_accuracy: 0.3120
Epoch 00003: early stopping


In [17]:
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis = 1)

In [19]:
from sklearn.metrics import classification_report
print(classification_report(test_encode_tags, y_pred))

              precision    recall  f1-score   support

           0       0.25      0.10      0.14        40
           1       0.46      0.55      0.50       132
           2       0.52      0.63      0.57       193
           3       0.71      0.54      0.62        46
           4       0.38      0.42      0.40       129
           5       0.54      0.52      0.53       116
           6       0.00      0.00      0.00        37

    accuracy                           0.48       693
   macro avg       0.41      0.39      0.39       693
weighted avg       0.46      0.48      0.47       693



c:\python\python36\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\python\python36\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\python\python36\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## 4.2 LSTM Model

In [13]:
from keras.models import Sequential
from keras.layers import Embedding, SpatialDropout1D, LSTM, Dense

model2 = Sequential()
model2.add(Embedding(vocab_size, embedding_dim, input_length=MAX_LEN))
# model2.add(SpatialDropout1D(0.2))
model2.add(LSTM(128))
model2.add(Dense(128, activation='sigmoid'))
model2.add(Dense(7, activation='softmax'))
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

NameError: name 'vocab_size' is not defined

In [33]:
y_train_lstm = tf.keras.utils.to_categorical(train_encode_tags)
y_valid_lstm = tf.keras.utils.to_categorical(valid_encode_tags)

history2 = model2.fit(X_train, y_train,
                    epochs = 20,
                    callbacks = [es],
                    validation_data=(X_valid, y_valid),
                    batch_size=64)

Epoch 1/20
87/87 [==============================] - 31s 355ms/step - loss: 1.8035 - accuracy: 0.2719 - val_loss: 1.7398 - val_accuracy: 0.3120
Epoch 2/20
87/87 [==============================] - 28s 318ms/step - loss: 1.7802 - accuracy: 0.2809 - val_loss: 1.7703 - val_accuracy: 0.3120
Epoch 00002: early stopping


## 4. BiLSTM model

In [14]:
bi_model = Sequential()
bi_model.add(Embedding(vocab_size, embedding_dim, input_length=MAX_LEN))
bi_model.add(SpatialDropout1D(0.2))
bi_model.add(layers.Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2)))
bi_model.add(Dense(7, activation='softmax'))
bi_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

NameError: name 'vocab_size' is not defined

In [24]:
bi_history = bi_model.fit(X_train, y_train,
                    epochs=10,
                    callbacks = [es],
                    validation_data=(X_valid, y_valid),
                    batch_size=32)

Epoch 1/10
174/174 [==============================] - 317s 2s/step - loss: 1.7038 - accuracy: 0.3270 - val_loss: 1.5055 - val_accuracy: 0.4329
Epoch 2/10
174/174 [==============================] - 356s 2s/step - loss: 1.2973 - accuracy: 0.5125 - val_loss: 1.3263 - val_accuracy: 0.5058
Epoch 3/10
174/174 [==============================] - 322s 2s/step - loss: 1.0063 - accuracy: 0.6387 - val_loss: 1.2990 - val_accuracy: 0.5175
Epoch 4/10
174/174 [==============================] - 296s 2s/step - loss: 0.8090 - accuracy: 0.7119 - val_loss: 1.3786 - val_accuracy: 0.5058
Epoch 00004: early stopping


In [42]:
bi_model.save("bi_model.h5")

NameError: name 'bi_model' is not defined

In [26]:
y_pred = bi_model.predict(X_test)
y_pred = np.argmax(y_pred, axis = 1)

In [27]:
from sklearn.metrics import classification_report
print(classification_report(test_encode_tags, y_pred))

              precision    recall  f1-score   support

           0       0.48      0.38      0.42        40
           1       0.49      0.64      0.55       132
           2       0.63      0.56      0.59       193
           3       0.61      0.61      0.61        46
           4       0.43      0.41      0.42       129
           5       0.52      0.64      0.57       116
           6       0.78      0.19      0.30        37

    accuracy                           0.53       693
   macro avg       0.56      0.49      0.50       693
weighted avg       0.55      0.53      0.53       693



## 4.3 Dense + TF IDF vectorize

In [47]:
model_3 = Sequential()
model_3.add(layers.Dense(512, activation = "relu"))
model_3.add(layers.Dense(128, activation = "sigmoid"))
model_3.add(layers.Dense(7, activation = "softmax"))
model_3.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ['accuracy'])

In [48]:
y_train = tf.keras.utils.to_categorical(train_encode_tags)
y_valid = tf.keras.utils.to_categorical(valid_encode_tags)

es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1)

history_3 = model_3.fit(X_train, y_train,
                    epochs=10,
                    callbacks = [es],
                    validation_data=(X_valid, y_valid),
                    batch_size=64)

Epoch 1/10
87/87 [==============================] - 2s 19ms/step - loss: 1.7517 - accuracy: 0.2751 - val_loss: 1.5560 - val_accuracy: 0.4621
Epoch 2/10
87/87 [==============================] - 1s 16ms/step - loss: 1.2672 - accuracy: 0.5596 - val_loss: 1.2702 - val_accuracy: 0.5175
Epoch 3/10
87/87 [==============================] - 1s 15ms/step - loss: 0.8541 - accuracy: 0.7184 - val_loss: 1.2993 - val_accuracy: 0.5277
Epoch 00003: early stopping


In [49]:
y_pred = model_3.predict(X_test)
y_pred = np.argmax(y_pred, axis = 1)

In [50]:
from sklearn.metrics import classification_report
print(classification_report(test_encode_tags, y_pred))

              precision    recall  f1-score   support

           0       0.38      0.15      0.21        40
           1       0.50      0.68      0.58       132
           2       0.58      0.59      0.58       193
           3       0.79      0.59      0.68        46
           4       0.43      0.48      0.46       129
           5       0.54      0.56      0.55       116
           6       1.00      0.08      0.15        37

    accuracy                           0.53       693
   macro avg       0.60      0.45      0.46       693
weighted avg       0.55      0.53      0.51       693



## 4.4 Linear SVC + TF IDF vectorize

In [34]:
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

In [36]:
linear_svc = LinearSVC()
clf = linear_svc.fit(X_train, train_encode_tags)

svc_model = CalibratedClassifierCV(base_estimator=linear_svc, cv="prefit")

svc_model.fit(X_train, train_encode_tags)
y_pred = svc_model.predict(X_test)

In [37]:
print(classification_report(test_encode_tags, y_pred))

              precision    recall  f1-score   support

           0       0.27      0.30      0.29        40
           1       0.51      0.54      0.53       132
           2       0.56      0.60      0.58       193
           3       0.64      0.63      0.64        46
           4       0.48      0.45      0.46       129
           5       0.55      0.54      0.55       116
           6       0.58      0.41      0.48        37

    accuracy                           0.52       693
   macro avg       0.51      0.49      0.50       693
weighted avg       0.53      0.52      0.52       693

